In [1]:
import os 
from pathlib import Path

root = Path('codegraphs/diversevul/')
old = root/Path('preprocessed_with_degreecount')
new = root/Path('preprocessed_v1_withdegreecount_allgraphs')
old_erik = root/Path('erik_preprocessed')


In [8]:
import torch
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor
import glob

# get average graph size in new
sizes = []

def load_graph(file):
    graph = torch.load(file)
    return graph.x.size(0)

with ThreadPoolExecutor() as executor:
    for i, result in enumerate(executor.map(load_graph, tqdm(new.glob('**/*.pt')))):
        sizes.append(result)
        avg = sum(sizes)/len(sizes)
        print(i, avg, end ='\r')

329735it [00:19, 17280.13it/s]


KeyboardInterrupt: 

In [10]:
# filter smaller 10 and bigger 1000
print(f'filtering {len(sizes)} graphs')
x = [s for s in sizes if s > 10 and s < 1000]

# print avg
print(f'avg: {sum(x)/len(x)}')

# filtering 225202 graphs
# avg: 124.21399027344029

filtering 225202 graphs
avg: 124.21399027344029


In [6]:
# import torch 
# def get_missing_file_graphsize(reference, missingfolder):
#     missing = set(os.listidr(root/reference)) - set(os.listdir(root/missingfolder))
#     avg = []
#     for m in missing:
#         s = torch.load(root/reference/m).x.size(0)
#         avg.append(s)
#         avergae = sum(avg)/len(avg)
#         print(f'avg, {avergae}')
        
import os
import torch
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm

def get_missing_file_graphsize(reference, missingfolder):
    print('missing')
    missing = set(os.listdir(reference)) - set(os.listdir(missingfolder))
    print('missing',len(missing))
    missng_form_other = set(os.listdir(missingfolder)) - set(os.listdir(reference))
    print('missing from other', len(missng_form_other))
    
    avg = []

    def load_graph(m):
        s = torch.load(os.path.join(reference, m)).x.size(0)
        return s

    with ThreadPoolExecutor() as executor:
        for i, result in enumerate(executor.map(load_graph, tqdm(missing))):
            avg.append(result)
            avergae = sum(avg)/len(avg)
            print(f'{i}  avg, {avergae}',end='\r')
    
    return avg

In [7]:
#missing files:

a = get_missing_file_graphsize(new,old)

missing


missing 81641
missing from other 39


100%|██████████| 81641/81641 [00:03<00:00, 26486.94it/s] 


In [10]:
len(a)

81641

In [16]:
# filter smaller 10 and bigger 1000

# 77864 smaller
# 3777 large filtered
# none between filtered
ax = [s for s in a if s > 10 and s < 1000]
print(len(ax))
print(f'avg: {sum(ax)/len(ax)}')

0


ZeroDivisionError: division by zero

# Test dot loading

In [198]:
list(Gagraph.nodes())

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11']

In [213]:
import torch
from torch_geometric.data import InMemoryDataset, Data, HeteroData
from torch_geometric.utils import to_edge_index
from tqdm import tqdm
from pathlib import Path
import pickle
import networkx as nx
from word2vec.word2vec import Word2VecBuilder
from multiprocessing import Pool, cpu_count
from functools import partial
import os 

import pygraphviz as pgv
import networkx as nx
from tqdm.auto import tqdm

config ={}
config['device'] = 'cuda'

label_categories = {}
# with open(Path(self.root)/'label_info.pkl', 'rb') as f:
    # labels = pickle.load(f)['label'].keys()
labels = ['TranslationUnitDeclaration', 'ConstructExpression','FunctionDeclaration', 'CompoundStatement', 'CallExpression', 'DeclaredReferenceExpression', 'WhileStatement', 'BinaryOperator', 'Literal', 'ReturnStatement', 'ParamVariableDeclaration', 'VariableDeclaration', 'DeclarationStatement', 'UnaryOperator', 'MemberExpression', 'ConditionalExpression', 'IfStatement', 'ForStatement', 'BreakStatement', 'MemberCallExpression', 'ArraySubscriptionExpression', 'TypeIdExpression', 'InitializerListExpression', 'ProblemDeclaration', 'CastExpression', 'ContinueStatement', 'EmptyStatement', 'GotoStatement', 'LabelStatement', 'SwitchStatement', 'CaseStatement', 'ProblemExpression', 'ExpressionList', 'DoStatement', 'DefaultStatement', 'RecordDeclaration', 'FieldDeclaration', 'ConstructorDeclaration', 'DesignatedInitializerExpression', 'EnumDeclaration', 'EnumConstantDeclaration', 'MethodDeclaration', 'ASMDeclarationStatement', 'TypedefDeclaration']
word2vec = Word2VecBuilder()        
sorted_labels = sorted(labels)
# onehot encode labels
for label in sorted_labels:
    onehot = torch.zeros(len(labels))
    onehot[sorted_labels.index(label)] = 1
    label_categories[label] = onehot
def column_normalize_dense_features_torch(mx):  
        '''Column-normalize dense features'''  # amos
        colsum = torch.sum(mx, axis=0)
        c_inv = torch.pow(colsum, -1)
        c_inv[torch.isinf(c_inv)] = 0.
        c_mat_inv = torch.diag(c_inv)
        mx = mx.mm(c_mat_inv)
        return mx
def get_adjacency_matrices(G):
        adj_matrices = {edge_type: torch.zeros((len(G.nodes), len(G.nodes)), dtype=torch.float32, device=config['device']) for edge_type in ['AST','CFG','CG','DFG']}
        node_list = [str(i)for i in sorted(int(s) for s in list(G.nodes()))]
        node_mapping = {x:i for i, x in enumerate(node_list)}
        print(node_mapping)
        # Update adjacency matrices based on edge types
        for edge in G.edges(data=True):
            source = edge[0]
            target = edge[1]
            edge_type = edge[2]['label']
            try:
                adj_matrices[edge_type][node_mapping[source]][node_mapping[target]] += 1  # Adjust indices to start from 0
            except KeyError:
                adj_matrices[edge_type.replace('"','')][node_mapping[source]][node_mapping[target]] = 1  # Adjust indices to start from 0

        return node_list, {edge_type: adj_matrix.to_sparse() for edge_type, adj_matrix in adj_matrices.items()}
    
    
def get_graph_nxpydot(file):
        #G = nx.drawing.nx_agraph.read_dot(file)
      
        G = nx.Graph(nx.drawing.nx_pydot.read_dot(file))
        
        # with open(file, "r", encoding="utf8") as f:
        #     G = nx.Graph(pgv.AGraph(f.read()))  
        
        # if not (len(G.nodes) <= 1000 and len(G.nodes) >= 10):  # filter graphs which are too small or too large
        #     placeholder_edgeindex = torch.tensor([[0,0],[0,0]], dtype=torch.long, device=  config['device'])
        #     placeholder_x = torch.tensor([[0,0]], dtype=torch.float32, device=  config['device'])
        #     return placeholder_edgeindex, placeholder_x 
        
        node_list, adj_matrices =   get_adjacency_matrices(G)

        # get binary adjacency matrix for all edge types
        matrix = torch.zeros((len(G.nodes), len(G.nodes)), layout=torch.sparse_coo, dtype=torch.float32, device=  config['device'])
        for adj_matrix in adj_matrices.values():
            matrix = matrix+adj_matrix

        # set nonzero to 1, undirected
        #matrix = matrix + matrix.T  # undirected graph
        coo = matrix.coalesce() 
        values = torch.ones(coo.values().shape).to(  config['device'])
        matrix = torch.sparse_coo_tensor(coo.indices(), values, matrix.shape, device=  config['device'])
        
        
        
        # get node degrees per edge type and in-/out-degrees
        # in1, in2, in3, out1 out2 out3, total1, total2, total3, total4...
        
        num_types = len(adj_matrices)
        degrees = torch.zeros((len(G.nodes), num_types*3+1), dtype=torch.float32, device=  config['device'])
        for i, adj_matrix in enumerate(adj_matrices.values()):
            degrees[:,i] = torch.sum(adj_matrix, dim=1).to_dense()  # out-degrees
            degrees[:,i+num_types] = torch.sum(adj_matrix, dim=0).to_dense()   # in-degrees
            degrees[:,i+2*num_types] = degrees[:,i] + degrees[:,i+num_types]  # total degrees
            
        degrees[:,-1] = torch.sparse.sum(matrix, dim=1).to_dense()   # total degrees (also total1, 2, 3 because sparse does not support range index)
        
        # add triangle counts per edge type and total triangle counts
        # with triangle count and degree count, we could calculate clustering coefficient
        triangle_counts = torch.zeros((len(G.nodes), num_types+1), device=  config['device'])
        for i, adj_matrix in enumerate(adj_matrices.values()):
            undirected_adj = (adj_matrix + adj_matrix.T).to_sparse()
            # sparse = undirected_adj.to_sparse()
            # unnormalized triangle count
            A_times_A = torch.sparse.mm(undirected_adj,undirected_adj)
            # sum(mul(A,B), dim=1) faster than trace(mm(A,B))
            triangle_counts[:,i] = torch.sparse.sum(torch.mul(undirected_adj, A_times_A), dim=1).to_dense()
        
        triangle_counts[:,-1] = torch.sum(triangle_counts[:,:-1], dim=1)
        
        # onehot label of node  e.g. "ParamVariableDeclaration"
        labels = [  label_categories[G.nodes[node]['label'].replace('"','')] for node in node_list]
        labels = torch.stack(labels).to(  config['device']).to(torch.float32)
        
        
        
        # can implement later:
        # or fasttext 
        # or SentenceBert
        # pagerank
        #
        # in, out, total * 4 (4 edge types) + total degree of all
        # degrees: 13, triangle_counts: 5
        X = torch.cat([degrees, triangle_counts], dim=1)
        X =   column_normalize_dense_features_torch(mx=X)
        
        node_embeddings = []
        # gensim word vectors (average for sentence vector)
        for node_id in node_list:
            average_embedding =   word2vec.get_embedding(G, node_id)
            average_embedding = torch.tensor(average_embedding, dtype=torch.float32, device=  config['device'])
            node_embeddings.append(average_embedding)
        
        node_embeddings = torch.stack(node_embeddings)
        
        X = torch.cat([X, labels], dim=1)
        
        # eye = torch.eye(len(G.nodes), device=  config['device']).to_sparse()
        
    
            
        return to_edge_index(matrix)[0], X  # only edge_index, not count
    
    
def get_graph_nxagraph( file):
        G = nx.drawing.nx_agraph.read_dot(file)
      
        # G = nx.Graph(nx.drawing.nx_pydot.read_dot(file))
        
        # with open(file, "r", encoding="utf8") as f:
        #     G = nx.Graph(pgv.AGraph(f.read()))  
        
        # if not (len(G.nodes) <= 1000 and len(G.nodes) >= 10):  # filter graphs which are too small or too large
        #     placeholder_edgeindex = torch.tensor([[0,0],[0,0]], dtype=torch.long, device=  config['device'])
        #     placeholder_x = torch.tensor([[0,0]], dtype=torch.float32, device=  config['device'])
        #     return placeholder_edgeindex, placeholder_x 
        
        node_list, adj_matrices =   get_adjacency_matrices(G)

        # get binary adjacency matrix for all edge types
        matrix = torch.zeros((len(G.nodes), len(G.nodes)), layout=torch.sparse_coo, dtype=torch.float32, device=  config['device'])
        for adj_matrix in adj_matrices.values():
            matrix = matrix+adj_matrix

        # set nonzero to 1, undirected
        #matrix = matrix + matrix.T  # undirected graph
        coo = matrix.coalesce() 
        values = torch.ones(coo.values().shape).to(  config['device'])
        matrix = torch.sparse_coo_tensor(coo.indices(), values, matrix.shape, device=  config['device'])
        
        
        
        # get node degrees per edge type and in-/out-degrees
        # in1, in2, in3, out1 out2 out3, total1, total2, total3, total4...
        
        num_types = len(adj_matrices)
        degrees = torch.zeros((len(G.nodes), num_types*3+1), dtype=torch.float32, device=  config['device'])
        for i, adj_matrix in enumerate(adj_matrices.values()):
            degrees[:,i] = torch.sum(adj_matrix, dim=1).to_dense()  # out-degrees
            degrees[:,i+num_types] = torch.sum(adj_matrix, dim=0).to_dense()   # in-degrees
            degrees[:,i+2*num_types] = degrees[:,i] + degrees[:,i+num_types]  # total degrees
            
        degrees[:,-1] = torch.sparse.sum(matrix, dim=1).to_dense()   # total degrees (also total1, 2, 3 because sparse does not support range index)
        
        # add triangle counts per edge type and total triangle counts
        # with triangle count and degree count, we could calculate clustering coefficient
        triangle_counts = torch.zeros((len(G.nodes), num_types+1), device=  config['device'])
        for i, adj_matrix in enumerate(adj_matrices.values()):
            undirected_adj = (adj_matrix + adj_matrix.T).to_sparse()
            # sparse = undirected_adj.to_sparse()
            # unnormalized triangle count
            A_times_A = torch.sparse.mm(undirected_adj,undirected_adj)
            # sum(mul(A,B), dim=1) faster than trace(mm(A,B))
            triangle_counts[:,i] = torch.sparse.sum(torch.mul(undirected_adj, A_times_A), dim=1).to_dense()
        
        triangle_counts[:,-1] = torch.sum(triangle_counts[:,:-1], dim=1)
        
        # onehot label of node  e.g. "ParamVariableDeclaration"
        labels = [  label_categories[G.nodes[node]['label'].replace('"','')] for node in node_list]
        labels = torch.stack(labels).to(  config['device']).to(torch.float32)
        
        
        
        # can implement later:
        # or fasttext 
        # or SentenceBert
        # pagerank
        #
        # in, out, total * 4 (4 edge types) + total degree of all
        # degrees: 13, triangle_counts: 5
        X = torch.cat([degrees, triangle_counts], dim=1)
        X =   column_normalize_dense_features_torch(mx=X)
        
        node_embeddings = []
        # gensim word vectors (average for sentence vector)
        for node_id in node_list:
            average_embedding =   word2vec.get_embedding(G, node_id)
            average_embedding = torch.tensor(average_embedding, dtype=torch.float32, device=  config['device'])
            node_embeddings.append(average_embedding)
        
        node_embeddings = torch.stack(node_embeddings)
        
        X = torch.cat([X, labels, node_embeddings], dim=1)
        
        # eye = torch.eye(len(G.nodes), device=  config['device']).to_sparse()
        
      
            
        return to_edge_index(matrix)[0], X  # only edge_index, not count
    
def get_graph_pgvagraph(file):
        #G = nx.drawing.nx_agraph.read_dot(file)
      
        # G = nx.Graph(nx.drawing.nx_pydot.read_dot(file))
        
        with open(file, "r", encoding="utf8") as f:
            G = nx.Graph(pgv.AGraph(f.read()))  
        
        # if not (len(G.nodes) <= 1000 and len(G.nodes) >= 10):  # filter graphs which are too small or too large
        #     placeholder_edgeindex = torch.tensor([[0,0],[0,0]], dtype=torch.long, device=  config['device'])
        #     placeholder_x = torch.tensor([[0,0]], dtype=torch.float32, device=  config['device'])
        #     return placeholder_edgeindex, placeholder_x 
        
        node_list, adj_matrices =   get_adjacency_matrices(G)

        # get binary adjacency matrix for all edge types
        matrix = torch.zeros((len(G.nodes), len(G.nodes)), layout=torch.sparse_coo, dtype=torch.float32, device=  config['device'])
        for adj_matrix in adj_matrices.values():
            matrix = matrix+adj_matrix

        # set nonzero to 1, undirected
        #matrix = matrix + matrix.T  # undirected graph
        coo = matrix.coalesce() 
        values = torch.ones(coo.values().shape).to(  config['device'])
        matrix = torch.sparse_coo_tensor(coo.indices(), values, matrix.shape, device=  config['device'])
        
        
        
        # get node degrees per edge type and in-/out-degrees
        # in1, in2, in3, out1 out2 out3, total1, total2, total3, total4...
        
        num_types = len(adj_matrices)
        degrees = torch.zeros((len(G.nodes), num_types*3+1), dtype=torch.float32, device=  config['device'])
        for i, adj_matrix in enumerate(adj_matrices.values()):
            degrees[:,i] = torch.sum(adj_matrix, dim=1).to_dense()  # out-degrees
            degrees[:,i+num_types] = torch.sum(adj_matrix, dim=0).to_dense()   # in-degrees
            degrees[:,i+2*num_types] = degrees[:,i] + degrees[:,i+num_types]  # total degrees
            
        degrees[:,-1] = torch.sparse.sum(matrix, dim=1).to_dense()   # total degrees (also total1, 2, 3 because sparse does not support range index)
        
        # add triangle counts per edge type and total triangle counts
        # with triangle count and degree count, we could calculate clustering coefficient
        triangle_counts = torch.zeros((len(G.nodes), num_types+1), device=  config['device'])
        for i, adj_matrix in enumerate(adj_matrices.values()):
            undirected_adj = (adj_matrix + adj_matrix.T).to_sparse()
            # sparse = undirected_adj.to_sparse()
            # unnormalized triangle count
            A_times_A = torch.sparse.mm(undirected_adj,undirected_adj)
            # sum(mul(A,B), dim=1) faster than trace(mm(A,B))
            triangle_counts[:,i] = torch.sparse.sum(torch.mul(undirected_adj, A_times_A), dim=1).to_dense()
        
        triangle_counts[:,-1] = torch.sum(triangle_counts[:,:-1], dim=1)
        
        # onehot label of node  e.g. "ParamVariableDeclaration"
        labels = [  label_categories[G.nodes[node]['label'].replace('"','')] for node in node_list]
        labels = torch.stack(labels).to(  config['device']).to(torch.float32)
        
        
        
        # can implement later:
        # or fasttext 
        # or SentenceBert
        # pagerank
        #
        # in, out, total * 4 (4 edge types) + total degree of all
        # degrees: 13, triangle_counts: 5
        X = torch.cat([degrees, triangle_counts], dim=1)
        X =   column_normalize_dense_features_torch(mx=X)
        
        node_embeddings = []
        # gensim word vectors (average for sentence vector)
        for node_id in node_list:
            average_embedding =   word2vec.get_embedding(G, node_id)
            average_embedding = torch.tensor(average_embedding, dtype=torch.float32, device=  config['device'])
            node_embeddings.append(average_embedding)
        
        node_embeddings = torch.stack(node_embeddings)
        
        X = torch.cat([X, labels, node_embeddings], dim=1)
        
        
        return to_edge_index(matrix)[0], X  # only edge_index, not count
    
get_graph_nxagraph(raw_file)[0]

{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9, '11': 10}
Loading wordvectors.
No enclosing found for  {'label': 'ParamVariableDeclaration', 'lexeme': 'unknown0', 'location': ' '} MultiDiGraph named 'G' with 11 nodes and 23 edges
No enclosing found for  {'label': 'ParamVariableDeclaration', 'lexeme': 'int1', 'location': ' '} MultiDiGraph named 'G' with 11 nodes and 23 edges


tensor([[ 0,  0,  1,  1,  1,  2,  2,  3,  3,  3,  3,  4,  4,  5,  5,  6,  7,  8,
          8,  8],
        [ 1,  8,  2,  4,  7,  3,  6,  4,  5,  6,  8,  5,  9,  3, 10,  1,  5,  3,
          9, 10]], device='cuda:0')

In [ ]:
# 74 with degree features, 67.9 bacc without degree features

In [244]:
# get small graph 
for file in os.listdir(old):
    data = torch.load(old/file)
    if data.x.size(0) < 15 and data.x.size(0)>12:
        print(file, data.x.size(0))
        break

6c4841c2b6c32a134f9f36e5e08857138cc12b10_321685412815739880486950646018483679442_0.cpg.pt 14


In [251]:
graph_old = torch.load('codegraphs/diversevul/preprocessed_with_degreecount/826bfe4bbd7594188e3d74d2539d9707b1c6a14b_45290177164699323617404875727428861719_0.cpg.pt')
graph_new= torch.load('codegraphs/diversevul/preprocessed_v1_withdegreecount_allgraphs/826bfe4bbd7594188e3d74d2539d9707b1c6a14b_45290177164699323617404875727428861719_0.cpg.pt')

In [271]:
graph_old.edge_index.shape, graph_new.edge_index.shape

(torch.Size([2, 29]), torch.Size([2, 18]))

tensor([[ 0,  0,  3,  3,  3,  3,  4,  4,  5,  5,  5,  5,  6,  6,  7,  7, 10, 10,
          0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
        [ 3, 10,  4,  6,  8,  9,  5,  8,  6,  7,  8, 10,  1,  7,  2,  9,  1,  2,
          0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]])

In [252]:
import pickle 
graph_erik = pickle.load(open('codegraphs/diversevul/erik_preprocessed/826bfe4bbd7594188e3d74d2539d9707b1c6a14b_45290177164699323617404875727428861719_0.cpg.pt','rb'))

In [253]:
raw_file = 'codegraphs/diversevul/raw/826bfe4bbd7594188e3d74d2539d9707b1c6a14b_45290177164699323617404875727428861719_0.cpg'

In [254]:
graph_erik['edge_index'].shape, graph_erik['num_nodes']

(torch.Size([2, 36]), 11)

In [176]:
g = from_networkx_multi(Gpydot)
g.edge_index

tensor([[ 0,  0,  1,  1,  1,  1,  1,  2,  2,  2,  3,  3,  3,  3,  3,  4,  4,  4,
          4,  5,  5,  5,  5,  6,  6,  6,  7,  7,  8,  8,  8,  8,  9,  9, 10, 10],
        [ 1,  8,  0,  4,  2,  7,  6,  1,  3,  6,  2,  6,  4,  5,  8,  1,  3,  5,
          9,  3,  4, 10,  7,  1,  2,  3,  1,  5,  0,  3,  9, 10,  4,  8,  5,  8]])

In [195]:
len(Gagraph.edges()), graph_new.edge_index.shape, graph_old.edge_index.shape

(23, torch.Size([2, 18]), torch.Size([2, 29]))

In [189]:
nx.DiGraph(Gagraph).edges()

OutEdgeView([('1', '2'), ('1', '9'), ('2', '3'), ('2', '5'), ('2', '8'), ('3', '4'), ('3', '7'), ('4', '5'), ('4', '6'), ('4', '7'), ('4', '9'), ('5', '6'), ('5', '10'), ('6', '4'), ('6', '11'), ('7', '2'), ('8', '6'), ('9', '4'), ('9', '10'), ('9', '11')])

In [183]:
g2 = from_networkx_multi(nx.DiGraph(Gagraph))

In [ ]:
  "1" -> "2" [label = "AST", properties = ""]
  "1" -> "9" [label = "AST", properties = ""]
  "2" -> "5" [label = "CFG", properties = ""]
  "2" -> "3" [label = "AST", properties = ""]
  "2" -> "8" [label = "AST", properties = ""]
  "3" -> "4" [label = "AST", properties = ""]
  "3" -> "7" [label = "AST", properties = ""]
  "4" -> "7" [label = "CFG", properties = ""]
  "4" -> "5" [label = "AST", properties = ""]
  "4" -> "6" [label = "AST", properties = ""]
  "4" -> "9" [label = "CG", properties = ""]
#   "4" -> "7" [label = "CFG", properties = ""]
#   "4" -> "5" [label = "AST", properties = ""]
#   "4" -> "6" [label = "AST", properties = ""]
  "5" -> "6" [label = "CFG", properties = ""]
  "5" -> "10" [label = "DFG", properties = ""]
  "6" -> "4" [label = "CFG", properties = ""]
  "6" -> "11" [label = "DFG", properties = ""]
  "7" -> "2" [label = "DFG", properties = ""]
  "8" -> "6" [label = "DFG", properties = ""]
  "9" -> "4" [label = "DFG", properties = ""]
  "9" -> "10" [label = "AST", properties = ""]

In [187]:
g2.edge_index

tensor([[ 0,  0,  1,  1,  1,  2,  2,  3,  3,  3,  3,  4,  4,  5,  5,  6,  7,  8,
          8,  8],
        [ 1,  8,  2,  4,  7,  3,  6,  4,  5,  6,  8,  5,  9,  3, 10,  1,  5,  3,
          9, 10]])

In [180]:
graph_new.edge_index.shape

torch.Size([2, 18])

In [160]:
def get_tuples_from_2xn_tensor(tensor):
    return list(zip(tensor[0].tolist(), tensor[1].tolist()))

In [161]:
graph_erik_edges = set(get_tuples_from_2xn_tensor(graph_erik['edge_index']))

In [162]:
graph_graphviz_edges = set(get_tuples_from_2xn_tensor(graph_new.edge_index))

In [163]:
len(set(graph_erik_edges) - set(graph_graphviz_edges))

29

In [142]:
len(set(graph_graphviz_edges)), len(switched_set)

(152, 304)

In [148]:
switched_set = set()
for tup in (set(graph_erik_edges)) - set(graph_graphviz_edges):
    switched_set.add((tup[1],tup[0]))

len(set(graph_erik_edges) - set(switched_set))

35

In [164]:
set(graph_graphviz_edges) - set(switched_set)

{(0, 3),
 (0, 10),
 (3, 4),
 (3, 6),
 (3, 8),
 (3, 9),
 (4, 8),
 (5, 6),
 (5, 7),
 (5, 8),
 (5, 10),
 (6, 1),
 (6, 7),
 (7, 9),
 (10, 1),
 (10, 2)}

In [172]:
from_networkx_multi(Gpydot)

Data(edge_index=[2, 36], label=[36], lexeme=[11], enclosing=[11], location=[11], num_nodes=11)

In [167]:
graph_new # with graphviz

Data(x=[11, 162], edge_index=[2, 18], y=[1])

In [166]:
graph_old # my approach

Data(x=[11, 162], edge_index=[2, 29], y=[1])

In [171]:
from_networkx_multi(Gagraph)

Data(edge_index=[2, 23], enclosing=[9], label=[23], lexeme=[9], location=[11], num_nodes=11)

In [115]:
data = from_networkx_multi(Gagraph)
data 

Data(edge_index=[2, 193], enclosing=[70], label=[193], lexeme=[69], location=[81], num_nodes=81)

In [119]:
from_networkx_multi(Gpgv)

Data(edge_index=[2, 304], enclosing=[70], label=[304], lexeme=[69], location=[81], num_nodes=81)

In [120]:
from_networkx_multi(Gpydot)

Data(edge_index=[2, 304], label=[304], lexeme=[81], enclosing=[81], location=[81], num_nodes=81)

In [118]:
import torch_geometric 
import numpy as np
def from_networkx_multi(G):
    # original code: https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/utils/convert.html#from_networkx
    G = nx.convert_node_labels_to_integers(G)
    G = G.to_directed() if not nx.is_directed(G) else G
    edge_index = torch.LongTensor(list(G.edges)).t().contiguous()
    if isinstance(G, (nx.MultiDiGraph, nx.MultiGraph)):
        edge_index = edge_index[0:2, :]
    data = {}
    for i, (_, feat_dict) in enumerate(G.nodes(data=True)):
        for key, value in feat_dict.items():
            data[str(key)] = [value] if i == 0 else data[str(key)] + [value]
            for key, item in data.items():
                try:
                    if type(item) is list and len(item) > 0 and type(item[0]) is np.ndarray:
                        item = np.stack(item)
                    data[key] = torch.tensor(item)
                except ValueError:
                    pass
   
    for i, (_, _, feat_dict) in enumerate(G.edges(data=True)):
        for key, value in feat_dict.items():
            if key =='properties':
                continue 
            data[str(key)] = [value] if i == 0 else data[str(key)] + [value]
            
            
    for key, item in data.items():
        try:
            # print(key)
            # print(item)
            if type(item) is list and len(item) > 0 and type(item[0]) is np.ndarray:
                item = np.stack(item)
            data[key] = torch.tensor(item)
        except ValueError:
            pass
    data['edge_index'] = edge_index.view(2, -1)
    data = torch_geometric.data.Data.from_dict(data)
    data.num_nodes = G.number_of_nodes()
    return data
 

In [220]:
Gagraph_mine.shape

torch.Size([2, 20])

In [219]:
Gpydot_mine.shape, Gpgv_mine.shape

(torch.Size([2, 18]), torch.Size([2, 18]))

In [229]:
Gagraph_mine

tensor([[ 0,  0,  1,  1,  1,  2,  2,  3,  3,  3,  3,  4,  4,  5,  5,  6,  7,  8,
          8,  8],
        [ 1,  8,  2,  4,  7,  3,  6,  4,  5,  6,  8,  5,  9,  3, 10,  1,  5,  3,
          9, 10]], device='cuda:0')

In [234]:
# Gagraph = nx.drawing.nx_agraph.read_dot(raw_file)
# Gpydot = nx.Graph(nx.drawing.nx_pydot.read_dot(raw_file))
Gpgv = nx.Graph(pgv.AGraph(open(raw_file, "r", encoding="utf8").read()))

In [231]:
(Gpgv_mine +1).T# 1,6   

tensor([[ 1,  2],
        [ 1,  9],
        [ 2,  3],
        [ 2,  5],
        [ 2,  7],
        [ 2,  8],
        [ 3,  4],
        [ 3,  7],
        [ 4,  5],
        [ 4,  6],
        [ 4,  7],
        [ 4,  9],
        [ 5,  6],
        [ 5, 10],
        [ 6,  8],
        [ 6, 11],
        [ 9, 10],
        [ 9, 11]], device='cuda:0')

In [242]:
Gpydot.edges()

EdgeView([('1', '2'), ('1', '9'), ('2', '5'), ('2', '3'), ('2', '8'), ('2', '7'), ('3', '4'), ('3', '7'), ('4', '7'), ('4', '5'), ('4', '6'), ('4', '9'), ('5', '6'), ('5', '10'), ('6', '11'), ('6', '8'), ('9', '10'), ('9', '11')])

In [240]:
Gpgv.edges()

EdgeView([('1', '2'), ('1', '9'), ('2', '3'), ('2', '5'), ('2', '8'), ('2', '7'), ('3', '4'), ('3', '7'), ('4', '5'), ('4', '6'), ('4', '7'), ('4', '9'), ('5', '6'), ('5', '10'), ('6', '11'), ('6', '8'), ('9', '10'), ('9', '11')])

In [264]:
Gagraph = nx.drawing.nx_agraph.read_dot(raw_file)
len(Gagraph.edges())
Gagraph.edges()

OutMultiEdgeDataView([('1', '2'), ('1', '9'), ('2', '3'), ('2', '5'), ('2', '8'), ('3', '4'), ('3', '7'), ('4', '5'), ('4', '5'), ('4', '6'), ('4', '6'), ('4', '7'), ('4', '7'), ('4', '9'), ('5', '6'), ('5', '10'), ('6', '4'), ('6', '11'), ('7', '2'), ('8', '6'), ('9', '4'), ('9', '10'), ('9', '11')])

In [279]:
graph_old.edge_index.T

tensor([[ 0,  3],
        [ 0, 10],
        [ 3,  4],
        [ 3,  6],
        [ 3,  8],
        [ 3,  9],
        [ 4,  5],
        [ 4,  8],
        [ 5,  6],
        [ 5,  7],
        [ 5,  8],
        [ 5, 10],
        [ 6,  1],
        [ 6,  7],
        [ 7,  2],
        [ 7,  9],
        [10,  1],
        [10,  2],
        [ 0,  0],
        [ 1,  1],
        [ 2,  2],
        [ 3,  3],
        [ 4,  4],
        [ 5,  5],
        [ 6,  6],
        [ 7,  7],
        [ 8,  8],
        [ 9,  9],
        [10, 10]])

In [ ]:
  "1" -> "2" [label = "AST", properties = ""]
  "1" -> "9" [label = "AST", properties = ""]
  "2" -> "5" [label = "CFG", properties = ""]
  "2" -> "3" [label = "AST", properties = ""]
  "2" -> "8" [label = "AST", properties = ""]
  "3" -> "4" [label = "AST", properties = ""]
  "3" -> "7" [label = "AST", properties = ""]
  "4" -> "7" [label = "CFG", properties = ""]
  "4" -> "5" [label = "AST", properties = ""]
  "4" -> "6" [label = "AST", properties = ""]
  "4" -> "9" [label = "CG", properties = ""] 
  "4" -> "7" [label = "CFG", properties = ""] # ommited since already there
  "4" -> "5" [label = "AST", properties = ""] # ommited 
  "4" -> "6" [label = "AST", properties = ""] # ommited
  "5" -> "6" [label = "CFG", properties = ""]
  "5" -> "10" [label = "DFG", properties = ""]
  "6" -> "4" [label = "CFG", properties = ""]
  "6" -> "11" [label = "DFG", properties = ""]
#   "7" -> "2" [label = "DFG", properties = ""] # it reads this as 2->7
  "8" -> "6" [label = "DFG", properties = ""]
  "9" -> "4" [label = "DFG", properties = ""] # ommits this although different label to 4->9
  "9" -> "10" [label = "AST", properties = ""]
  "9" -> "11" [label = "AST", properties = ""]

In [277]:
graph_old.edge_index

tensor([[ 0,  0,  3,  3,  3,  3,  4,  4,  5,  5,  5,  5,  6,  6,  7,  7, 10, 10,
          0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
        [ 3, 10,  4,  6,  8,  9,  5,  8,  6,  7,  8, 10,  1,  7,  2,  9,  1,  2,
          0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]])

In [233]:
Gagraph = nx.drawing.nx_agraph.read_dot(raw_file)
Gpydot = nx.Graph(nx.drawing.nx_pydot.read_dot(raw_file))
Gpgv = nx.Graph(pgv.AGraph(open(raw_file, "r", encoding="utf8").read()))

len(Gagraph.edges()), len(Gpydot.edges()), len(Gpgv.edges())

(23, 18, 18)

In [214]:
Gagraph_mine = get_graph_nxagraph(raw_file)[0]
Gpydot_mine = get_graph_nxpydot(raw_file)[0]
Gpgv_mine = get_graph_pgvagraph(raw_file)[0]

{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9, '11': 10}
No enclosing found for  {'label': 'ParamVariableDeclaration', 'lexeme': 'unknown0', 'location': ' '} MultiDiGraph named 'G' with 11 nodes and 23 edges
No enclosing found for  {'label': 'ParamVariableDeclaration', 'lexeme': 'int1', 'location': ' '} MultiDiGraph named 'G' with 11 nodes and 23 edges
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9, '11': 10}
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9, '11': 10}
No enclosing found for  {'label': 'ParamVariableDeclaration', 'lexeme': 'unknown0', 'location': ' '} Graph named 'G' with 11 nodes and 18 edges
No enclosing found for  {'label': 'ParamVariableDeclaration', 'lexeme': 'int1', 'location': ' '} Graph named 'G' with 11 nodes and 18 edges


In [45]:
# compare graphs 
newfiles = os.listdir(new)
oldfiles = os.listdir(old)

overlap = set(newfiles) & set(oldfiles) 
i = 0
for file in overlap:
    newone = torch.load(new/file)
    oldone = torch.load(old/file)
    
    # newx = newone.x[:,:18]
    # oldx = oldone.x[:,:18]
    newedge = newone.edge_index
    oldedge = oldone.edge_index
    
    print(file,':')
    print('edgeindex sizes from files')
    print('graphviz:')
    print(newedge.shape)
    print('nxagraph:')
    print(oldedge.shape)
    
    rawfile = root/Path('raw')/file.replace('.pt','')
    print('load fresh')
    
    print('nxagraph:',{get_graph_nxagraph(rawfile)[0].shape})
    print('nxpydot:',{get_graph_nxpydot(rawfile)[0].shape})
    print('pvgagraph', {get_graph_pgvagraph(rawfile)[0].shape})
    
    if i ==10:
        break 
    i+=1

edc032b5e3f3ebb1006a9c89e0ae00504f47966f_199207534258974476555222615779136970485_0.cpg.pt :
edgeindex sizes from files
graphviz:
torch.Size([2, 261])
nxagraph:
torch.Size([2, 405])
load fresh
No enclosing found for  {'label': 'ParamVariableDeclaration', 'lexeme': 'bignumPtr0', 'location': ' '} MultiDiGraph named 'G' with 144 nodes and 429 edges
No enclosing found for  {'label': 'ParamVariableDeclaration', 'lexeme': 'bignumPtr0', 'location': ' '} MultiDiGraph named 'G' with 144 nodes and 429 edges
No enclosing found for  {'label': 'ParamVariableDeclaration', 'lexeme': 'charPtr0', 'location': ' '} MultiDiGraph named 'G' with 144 nodes and 429 edges
No enclosing found for  {'label': 'ParamVariableDeclaration', 'lexeme': 'unknown1', 'location': ' '} MultiDiGraph named 'G' with 144 nodes and 429 edges
No enclosing found for  {'label': 'ParamVariableDeclaration', 'lexeme': 'charPtr0', 'location': ' '} MultiDiGraph named 'G' with 144 nodes and 429 edges
No enclosing found for  {'label': 'Para

In [24]:
file

'edc032b5e3f3ebb1006a9c89e0ae00504f47966f_199207534258974476555222615779136970485_0.cpg.pt'